In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import helpers.Utilidades as utilidades
import helpers.Preprocessors as pp

In [2]:
dataset = pd.read_csv("./data/dataset.csv")
dataset.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium


In [3]:
columnasNumericasDiscretas, columnasNumericasContinuas, columnasCategoricas, columnasFecha = utilidades.ObtenerTiposColumnas(dataset)

In [4]:
y = "DEALSIZE"
columnasCategoricas.remove(y)
columnasCategoricas.remove("ORDERDATE")
columnasCategoricas.remove("TERRITORY")
columnasCategoricas.remove("ADDRESSLINE2")
columnasCategoricas.remove("STATE")
columnasNumericasDiscretas.remove("ORDERLINENUMBER")
columnasNumericasContinuas.remove("ORDERNUMBER")

# FEATURE ENGINEERING

## Variables Categoricas

In [5]:
dataset[columnasCategoricas].isnull().mean()

STATUS              0.000000
PRODUCTLINE         0.000000
PRODUCTCODE         0.000000
CUSTOMERNAME        0.000000
PHONE               0.000000
ADDRESSLINE1        0.000000
CITY                0.000000
POSTALCODE          0.026922
COUNTRY             0.000000
CONTACTLASTNAME     0.000000
CONTACTFIRSTNAME    0.000000
dtype: float64

## Varabiles Númericas Discretas

In [6]:
dataset[columnasNumericasDiscretas].isnull().mean()

QTR_ID      0.0
MONTH_ID    0.0
YEAR_ID     0.0
dtype: float64

## Variables Númericas Continuas 

In [7]:
dataset[columnasNumericasContinuas].isnull().mean()

QUANTITYORDERED    0.0
PRICEEACH          0.0
SALES              0.0
MSRP               0.0
dtype: float64

## Pipeline

In [8]:
def instanciatePipeline(df, y):
    columnasNumericasDiscretas, columnasNumericasContinuas, columnasCategoricas, columnasFecha = utilidades.ObtenerTiposColumnas(df)
    
    columnasCategoricas.remove(y)
    columnasCategoricas.remove("ORDERDATE")
    columnasCategoricas.remove("TERRITORY")
    columnasCategoricas.remove("ADDRESSLINE2")
    columnasCategoricas.remove("STATE")
    columnasNumericasDiscretas.remove("ORDERLINENUMBER")
    columnasNumericasContinuas.remove("ORDERNUMBER")

    bankChurner_Pipeline = Pipeline(steps=[
        ('frequency-encoder',
            pp.CategoricalEncoderOperator(columnasCategoricas)),
        ('CCA-encoder',
            pp.CategoricalCCAEnconderOperator("POSTALCODE")),
        ('outlier-treatment',
            pp.OutliersTreatmentOperator(columnas= columnasNumericasContinuas))
        #('target-enconder',
         #   pp.TargetEncoderOperator(y))
    ])

    return bankChurner_Pipeline

In [12]:
X = dataset.drop(['ORDERNUMBER', 'ORDERLINENUMBER', 'ORDERDATE', 'TERRITORY', 'ADDRESSLINE2', 'STATE', 'DEALSIZE'], axis=1)
y = dataset['DEALSIZE']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=2022)
dfSalida = instanciatePipeline(dataset, 'DEALSIZE').fit_transform(X_train, y_train)
mapperY = y.value_counts().to_dict()
dfSalida["DEALSIZE"] = y.map(mapperY)

In [13]:
dfSalida.to_csv("./data/fe_dataset.csv")